In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Analyse dataset
display(dataset.sample(5))
display(dataset.info())
display(dataset.describe())

In [ ]:
# Dropping duplicates
dataset = dataset.drop_duplicates()

In [ ]:
# Checking for statistical biases
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2, 2, figsize=(8,5))
ax1.boxplot(dataset["age"])
ax1.set_title('age')
ax2.boxplot(dataset["bmi"])
ax2.set_title('bmi')
ax3.hist(dataset["age"])
ax4.hist(dataset["bmi"]);

In [ ]:
# Defining features X and label y
X = dataset.drop(columns=['expenses'])
y = dataset['expenses']

In [ ]:
# Checking which categorical features are binary
display(X.sex.value_counts())
# smokers seem to be over-represented, need to get back to this after checking for "resampling strategies for imbalanced datasets"
display(X.smoker.value_counts())
display(X.region.value_counts())

In [ ]:
# sex and smoker are binary and will be converted to zeros and ones
from sklearn.preprocessing import OneHotEncoder
display(test_dataset.head)
binary_encoder = OneHotEncoder(sparse=False, drop='first')
X['sex'], X['smoker'] = binary_encoder.fit_transform(X[['sex', 'smoker']]).T
X.head()

In [ ]:
# region has 4 categories
display(dataset['region'].unique())
display(test_dataset.head)

cat_encoder = OneHotEncoder(sparse = False) # Instanciate encoder
X["ne"], X["nw"], X['se'], X['sw'] = cat_encoder.fit_transform(X[['region']]).T

X.head()

In [ ]:
X = X.drop(columns='region')

In [ ]:
#  Just to get an idea of results lets use a ML linear regression model.
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

# Instanciating the model
model = LinearRegression()

# Cross validating, fitting the model and getting score results
cross_val_results = cross_validate(model, X, y, cv=10,
                            scoring=['neg_mean_absolute_error'])
cross_val_results['test_neg_mean_absolute_error'].mean()*-1 # result is not ideal...


In [ ]:
# spliting into train test sets
from sklearn.model_selection import train_test_split
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[train_dataset.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  model.compile(loss='mse',
                optimizer='adam',
                metrics=['mae', 'mse'])
  return model

model = build_model()
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=100, restore_best_weights=True)

history = model.fit(train_dataset, train_labels,
                    epochs=1000,
                    validation_split = 0.2,
                    verbose=0,
                    callbacks=[es])

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)


In [ ]:
_patient_900 = test_dataset[:1]
predict_expenses_patient_900 = model.predict(_patient_900)
display("Predicted Expenses: ", predict_expenses_patient_900[0,0])
display("Real Expenses", test_labels[900])